# Multivariate Newton method

*[Written by: Maxime Pierre, 2023]* \
We now know from the basic idea behind Newton's method and how to implement it simply in `python`, in the case of function of a real variable. \
However, problems in continuum mechanics often involve systems of equations and/or vector/tensor variables. In this tutorial, we will walk through a simple 2D case: 2 unknowns, 2 equations. \
We will consider two planar curves, of following equations:
$$ 
(E) : \left\{
\begin{array}{}
f_1(x_1,x_2) = \left(x_1-\frac{1}{2}\right)^2 + 2\left(x_2+\frac{1}{2}\right)^2 - 1, \\
f_2(x_1,x_2) = \cos(x_2)x_1^2 + \cos(x_1)x_2^2 - \frac{1}{2}.
\end{array}
\right.
$$
$f_1$ is the equation to an ellipse and $f_2$ to an ellipse modified with a cosinus factor. Solving this system of equations amounts to finding the intersection between these two ellipse-like curves in the $(x_1,x_2)$ plane. \
Let us first define the two functions $f_1$ and $f_2$ and plot the two curves. For this, we will use a `contour` plot and display only the contour corresponding to a value of the function equal to 0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f1(x1, x2):
    return ### Exercise: write f_1 ###

def f2(x1, x2):
    return ### Exercise: write f_2 ###

fig = plt.figure()
gc = fig.gca()
x1 = np.linspace(-1.5, 1.5, 50)
x2 = np.linspace(-1.5, 1.5, 50)
X1, X2 = np.meshgrid(x1, x2)
gc.contour(x1, x2, f1(X1, X2), [0])
gc.contour(x1, x2, f2(X1, X2), [0])
gc.set_xlabel(r"$x_1$")
gc.set_ylabel(r"$x_2$")

As we can see, the two curves intersect in two locations. Let us see if we can find one of these intersections using Newton's method. Rather than scalar values, we will now deal with a vector unknown:
$$ X = \begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}.
$$
Our residue can also be seen as a vector:
$$
F(X) = \begin{bmatrix}
f_1(X) \\
f_2(X)
\end{bmatrix}.
$$


In [ ]:
def F(X): # Defining the residu and unknown as vectors
    x1, x2 = X
    return ### Exercise: return the right numpy vector ###

Now, we have to have an equivalent of the derivative of $F$. In function of two variables, the differential of the function is defined as such:
$$
g : (x_1, x_2) \rightarrow g(x_1, x_2),
$$
$$
\mathrm{d}g = \frac{\partial g}{\partial x_1}\mathrm{d}x_1 + \frac{\partial g}{\partial x_2}\mathrm{d}x_2,
$$
which can be seen as the following dot product:
$$
\mathrm{d}g =
\begin{bmatrix}
\frac{\partial g}{\partial x_1} & \frac{\partial g}{\partial x_2}
\end{bmatrix}
\cdot
\begin{bmatrix}
\mathrm{d}x_1 \\
\mathrm{d}x_2
\end{bmatrix}
$$
For our case, we have a system consisting of two equations. We could abusively write:
$$
\mathrm{d}F = \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} \\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2}
\end{bmatrix}
\cdot
\begin{bmatrix}
\mathrm{d}x_1 \\
\mathrm{d}x_2
\end{bmatrix}=
J\cdot \mathrm{d}X,
$$
where $J$ is called the jacobian matrix of the system, playing the same role as the derivative in the 1D case. \
Note that generalization to $n$ dimensions is quite straightforward:
$$
J = \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \cdots & \frac{\partial f_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_n}{\partial x_1} & \cdots &\frac{\partial f_n}{\partial x_n}
\end{bmatrix}=
\left( \frac{\partial f_i}{\partial x_j} \right)_{(i,j)}.
$$
Let us implement the jacobian matrix of our system:

In [ ]:
def J(X):
    x1, x2 = X

    ### Exercise: calculate the partial derivatives
    df1_dx1 = #
    df1_dx2 = #
    df2_dx1 = #
    df2_dx2 = #
    ###
    return np.array([[df1_dx1, df1_dx2], [df2_dx1, df2_dx2]])

Continuing our parallels with the 1D method, the iteration will be the following:
$$
X_{n+1} = X_n - J^{-1}(X_n)\cdot F(X_n).
$$
This of course requires $J(X_n)$ to be non-singular, just as the 1D method required the derivative to be non-zero. \
Evaluation of the convergence will be done through the norm of the residu, with the following convergence criterion for a precision $\varepsilon > 0$:
$$
\| F(X_n)\| \leq \varepsilon
$$

In [ ]:
def newton_method(residu, jacobian, initial_guess, tolerance, max_iteration=50):
    X_n = initial_guess
    X = [initial_guess]
    iteration = 0
    
    while np.linalg.norm(residu(X_n)) > tolerance and iteration < max_iteration:
        
        ### Exercise: update X_n ###
        
        X.append(X_n.copy())
        iteration += 1
        print("Iteration {}: X_{} = {}, residual norm = {}".format(iteration, iteration, X_n, np.linalg.norm(residu(X_n))))
        
    if np.linalg.norm(residu(X_n)) > tolerance: # Not converged
        print("Did not converge after {} iterations.".format(iteration))
    else: # Converged
        print("Converged in {} iterations.".format(iteration))
        
    return np.array(X)

Let us test the algorithm with the following initial guess:
$$
X_0 = \begin{bmatrix}
1 \\
1
\end{bmatrix},
$$
and a tolerance of $10^{-12}$:

In [ ]:
X_0 = [1,1]
eps = 1e-12

result = newton_method(F, J, X_0, eps)

gc.scatter((result.T)[0], (result.T)[1], c='r')
for i in range(len((result.T)[0])):
    gc.annotate( r"$X_{}$".format(i), (result[i,0], result[i,1]), c='r' )
fig

In [ ]:
X_0 = [0.5,0.5]

result = newton_method(F, J, X_0, eps)

gc.scatter((result.T)[0], (result.T)[1], c='b')
for i in range(len((result.T)[0])):
    gc.annotate( r"$X_{}$".format(i), (result[i,0], result[i,1]), c='b' )
fig

## Bonus: Newton's method with numerical approximation of the jacobian

Calculating the Jacobian matrix of the system can be pretty tedious at times (especially later when dealing with tensorial equations for instance). An alternative way consists in using an numerical approximate of the jacobian obtained by a small perturbation $\eta$. The partial derivatives, components of the jacobian matrix can be approximated as: \
$$
\frac{\partial f_i}{\partial x_j}\left( (\tilde{x}_k)_{1\leq k \leq n}\right) \simeq \frac{ f_i( (\tilde{x_k} + \delta_{jk}\eta)_{1\leq k\leq n}) - f_i( (\tilde{x_k} - \delta_{jk}\eta)_{1\leq k\leq n})}{ 2\eta },
$$
where $\delta_{jk}$ is Kronecker's delta. Let us implement it for our previous case and test it with $x_0 = [1,1]$.

In [ ]:
def numerical_J(X, residu, perturbation):
    ### Exercise: write a function which returns the numerical jacobian

X_0 = [1,1]
eta = 1e-9 # Perturbation magnitude for the numerical jacobian

result = newton_method(F, lambda X : numerical_J(X, F, eta), X_0, eps)

If we look closely at the residual value, we can see that we lost some precision compared to the exact jacobian. However, it did not affect our convergence overall.